In [ ]:
# ==============================
# Install packages (Colab)
# ==============================
!pip install -q tensorflow tensorflow-datasets streamlit pillow

# ==============================
# Imports
# ==============================
import tensorflow as tf
import tensorflow_datasets as tfds
from collections import Counter
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Optional: for Colab drive
from google.colab import drive
drive.mount('/content/drive')

# ==============================
# Load PlantVillage dataset
# ==============================
dataset, info = tfds.load("plant_village", with_info=True, as_supervised=True)
train_ds_full = dataset['train']

NUM_CLASSES = info.features['label'].num_classes  # 38 classes
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

print(f"Number of classes: {NUM_CLASSES}")
print(f"Class names: {info.features['label'].names}")

# ==============================
# Preprocessing
# ==============================
def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds_full = train_ds_full.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# ==============================
# Train / Validation split
# ==============================
total_examples = info.splits["train"].num_examples
train_size = int(0.8 * total_examples)
val_size = total_examples - train_size

train_ds = train_ds_full.take(train_size).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = train_ds_full.skip(train_size).take(val_size).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ==============================
# Compute class weights
# ==============================
counter = Counter()
for images, labels in train_ds:
    for l in labels.numpy():
        counter[int(l)] += 1

total_samples = sum(counter.values())
class_weights = {cls: total_samples/count for cls, count in counter.items()}

print("Class weights:", class_weights)

# ==============================
# Data augmentation
# ==============================
AUTOTUNE = tf.data.AUTOTUNE

def augment(images, labels):
    images = tf.image.random_flip_left_right(images)
    images = tf.image.random_flip_up_down(images)
    images = tf.image.random_brightness(images, 0.1)
    return images, labels

train_data_aug = train_ds.map(augment, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

# ==============================
# Build MobileNetV2 Functional Model
# ==============================
inputs = Input(shape=(128,128,3))
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
base_model.trainable = False  # Freeze base

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ==============================
# Callbacks
# ==============================
model = tf.keras.models.load_model('/content/drive/MyDrive/plant_disease_detection(0).keras')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/plant disease models/plant_disease_detection(1).keras',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# ==============================
# Train top layers
# ==============================
history = model.fit(
    train_data_aug,
    validation_data=val_ds,
    epochs=200,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr, checkpoint]
)

# ==============================
# Fine-tuning last 20 layers of MobileNetV2
# ==============================
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_data_aug,
    validation_data=val_ds,
    epochs=200,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr, checkpoint]
)

# ==============================
# Save model
# ==============================
model.save("plant_disease_detection_model.keras")
print("✅ Model saved successfully")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.6 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,933,350 (11.19 MB)

 Trainable params: 675,366 (2.58 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.9786 - loss: 2.0771
Epoch 1: val_loss improved from inf to 0.16722, saving model to /content/drive/MyDrive/plant disease models/plant_disease_detection(1).keras
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 562s 410ms/step - accuracy: 0.9786 - loss: 2.0771 - val_accuracy: 0.9550 - val_loss: 0.1672 - learning_rate: 3.1250e-05
Epoch 2/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.9794 - loss: 1.9763
Epoch 2: val_loss did not improve from 0.16722
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 533s 389ms/step - accuracy: 0.9794 - loss: 1.9764 - val_accuracy: 0.9551 - val_loss: 0.1674 - learning_rate: 3.1250e-05
Epoch 3/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - accuracy: 0.9764 - loss: 2.1413
Epoch 3: val_loss improved from 0.16722 to 0.16719, saving model to /content/drive/MyDrive/plant disease models/plant_disease_detection(1).keras
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 510s 375ms/step - accuracy: 0.9764 - loss: 2.1413 - val_accur